In [1]:
import pandas as pd

## Data

In [2]:
data = pd.read_excel('channel spend.xlsx')
data

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,2020-01-24 21:44:16
1,facebook,facebook,2020-03-01 15:44:35
2,organic,facebook,2019-12-07 16:37:06
3,search,tv,2020-01-27 16:09:32
4,discovery,youtube,2019-10-05 12:57:07
...,...,...,...
227623,facebook,facebook_organic,2019-11-17 14:12:33
227624,google_organic,referral,2019-12-06 18:02:13
227625,facebook,facebook,2019-12-21 19:40:44
227626,facebook,facebook,2020-01-17 23:58:51


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227628 entries, 0 to 227627
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   attribution_technical  227628 non-null  object        
 1   attribution_survey     224984 non-null  object        
 2   account_creation_date  227628 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 5.2+ MB


In [4]:
channel_spend = pd.read_csv('channel spend.csv',index_col=0)
channel_spend

,bing,display,facebook,search,youtube
201907,400,12,9000,13000,90
201908,900,20,13000,18500,180
201909,1000,13,10500,19000,100
201910,1100,19,17000,24000,130
201911,1300,29,23000,25000,550
201912,300,31,16000,38000,900
202001,2100,148,11000,41000,4360
202002,3700,94,14000,44000,2420


## Reserve only date and month

In [5]:
data['account_creation_date'] = data['account_creation_date'].dt.date
data['account_creation_date'] = (pd.DatetimeIndex(data['account_creation_date']).year)*100 + (pd.DatetimeIndex(data['account_creation_date']).month)
data['account_creation_date']

0         202001
1         202003
2         201912
3         202001
4         201910
           ...  
227623    201911
227624    201912
227625    201912
227626    202001
227627    201911
Name: account_creation_date, Length: 227628, dtype: int64

In [6]:
data

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,202001
1,facebook,facebook,202003
2,organic,facebook,201912
3,search,tv,202001
4,discovery,youtube,201910
...,...,...,...
227623,facebook,facebook_organic,201911
227624,google_organic,referral,201912
227625,facebook,facebook,201912
227626,facebook,facebook,202001


## Channel spend - attribution_survey

In [7]:
channel_list = ['bing', 'display', 'facebook', 'search', 'youtube']
time_list = [201907,201908,201909,201910,201911,201912,202001,202002]

In [8]:
# Create empty dataframe
result = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [9]:
for c in channel_list:
    count_list = []
    for t in time_list:
        count = 0
        for i in range(len(data)):
            if data.iloc[i]['account_creation_date'] == t:
                a = data.iloc[i]['attribution_survey']
                b = data.iloc[i]['attribution_technical']
                if (a in channel_list) and (b in channel_list):
                    if a==c:
                        count += 0.4
                    if b==c:
                        count += 0.6
                    continue
                elif a==c and (b not in channel_list):
                    count += 1
                    continue
                elif b==c and (a not in channel_list):
                    count += 1
                    continue
                    
        count_list.append(count)
    result[c] = count_list

result

,bing,display,facebook,search,youtube
201907,98.4,34.4,16039.6,1421.2,73.4
201908,57.4,53.6,16391.2,2033.6,247.2
201909,66.8,9.6,12269.4,1948.2,197.0
201910,68.8,40.8,14434.8,2301.6,260.0
201911,77.8,70.4,18462.2,2448.0,290.6
201912,24.4,89.2,14341.0,3636.0,233.4
202001,144.0,422.0,10572.8,4069.4,736.8
202002,247.0,267.0,12872.4,4422.4,519.2


In [10]:
spend_list = []
for c in channel_list:
    spend_list.append(channel_spend[c].sum())

spend_list

[10800, 366, 113500, 222500, 8730]

In [11]:
convert_list = []
for c in result:
    convert_list.append(result[c].sum())

convert_list

[784.5999999999992,
 987.000000000002,
 115383.39999999845,
 22280.399999999692,
 2557.5999999999995]

### Average CAC

In [12]:
avg_cac_list = []
for i in range(0,5):
    if convert_list[i] == 0:
        avg_cac_list.append(0)
    else:
        avg_cac = round(spend_list[i]/convert_list[i],2)
        avg_cac_list.append(avg_cac)

avg_cac_list

[13.76, 0.37, 0.98, 9.99, 3.41]

### Marginal CAC

In [13]:
# Create empty dataframe
marg_cac = pd.DataFrame(columns=['bing', 'display', 'facebook', 'search', 'youtube'],index=time_list)

In [14]:
for c in channel_list:
    marg_cac_list = []
    for t in time_list:
        if result[c][t] == 0:
            marg_cac_list.append(0)
        else:
            marg_cac_value = round(channel_spend[c][t]/result[c][t],2)
            marg_cac_list.append(marg_cac_value)
    marg_cac[c] = marg_cac_list

marg_cac

,bing,display,facebook,search,youtube
201907,4.07,0.35,0.56,9.15,1.23
201908,15.68,0.37,0.79,9.10,0.73
201909,14.97,1.35,0.86,9.75,0.51
201910,15.99,0.47,1.18,10.43,0.50
201911,16.71,0.41,1.25,10.21,1.89
201912,12.30,0.35,1.12,10.45,3.86
202001,14.58,0.35,1.04,10.08,5.92
202002,14.98,0.35,1.09,9.95,4.66
